# Урок 9. Языковое моделирование

## Задание

* Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/data/gb_NLP_les9_data/evgenyi_onegin (2).txt'

with open(path, 'rb') as file:
  text = file.read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [3]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [4]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [5]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [9]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [11]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

## LSTM модель

In [39]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [40]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [41]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [42]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 128)           16768     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          4722688   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_4 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense_3 (Dense)              (64, None, 131)           134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
__________________________________________

In [16]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 0.00050865, -0.00072038, -0.00021948, ...,  0.00095393,
        -0.00020139, -0.00040815],
       [ 0.00155357, -0.00186443, -0.00045575, ...,  0.00222344,
        -0.00039067, -0.00081843],
       [ 0.0028675 , -0.00303578, -0.00069807, ...,  0.00335905,
        -0.00053718, -0.00101423],
       ...,
       [ 0.00845018, -0.0050537 , -0.00362825, ...,  0.00520635,
        -0.00310409,  0.00022997],
       [ 0.00849199, -0.00505013, -0.00362504, ...,  0.00519917,
        -0.00311358,  0.00022165],
       [ 0.00852198, -0.00504748, -0.00362154, ...,  0.00519454,
        -0.00311925,  0.00021463]], dtype=float32)>

In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [18]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '    Как листья осенью гнилой.\n                        Несносно видеть пред собою\n                   '

Next Char Predictions: 
 'ЧкЛВ,CMМcщp DЬNИжэ\'СMРвv?ГцmцkaКzеЦumтУ{ИCТюD,ОкЦ!Lзe}qNГ"УэТбЖ}0СcEЛfvP--mш\'hQжerОшV65(ЬЕHХ5АщмFOиД'


In [19]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8769684


In [43]:
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

## Обучение

In [44]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
44/44 [==============================] - 38s 735ms/step - loss: 2.5156
Epoch 2/20
44/44 [==============================] - 33s 746ms/step - loss: 1.9652
Epoch 3/20
44/44 [==============================] - 33s 750ms/step - loss: 1.7432
Epoch 4/20
44/44 [==============================] - 34s 766ms/step - loss: 1.6369
Epoch 5/20
44/44 [==============================] - 33s 747ms/step - loss: 1.5282
Epoch 6/20
44/44 [==============================] - 33s 746ms/step - loss: 1.4660
Epoch 7/20
44/44 [==============================] - 33s 747ms/step - loss: 1.4320
Epoch 8/20
44/44 [==============================] - 33s 746ms/step - loss: 1.3972
Epoch 9/20
44/44 [==============================] - 33s 746ms/step - loss: 1.3717
Epoch 10/20
44/44 [==============================] - 34s 767ms/step - loss: 1.3726
Epoch 11/20
44/44 [==============================] - 33s 748ms/step - loss: 1.3553
Epoch 12/20
44/44 [==============================] - 33s 746ms/step - loss: 1.3478
Epoch 13/20
4

In [45]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_16'

In [46]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [47]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 128)            16768     
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           4722688   
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_8 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_4 (Dense)              (1, None, 131)            134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
__________________________________________

In [36]:
def generate_text(model, start_string, temperature = 0.5):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

## Подбор параметра temperature

In [48]:
for temperature in np.arange(0.1, 1.5, 0.1):
  text_ = generate_text(model, start_string=u"Вниманье дружбы возлюбя ", temperature=temperature)
  print(f"temperature={temperature}")
  print(text_)
  print(f"{'=' * 50}")

temperature=0.1
Вниманье дружбы возлюбя стет,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
temperature=0.2
Вниманье дружбы возлюбя стет,
                                                                                                                                                                                                                                                                                                                                                                                                                             

## Вывод

Сгенерированный текст наиболее похож на "текст" при temperature=0.9